<a href="https://colab.research.google.com/github/BhaveshKhaple/NLQ-2-SQL/blob/main/4th_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers datasets evaluate

import json
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset, DatasetDict
import evaluate

metric = evaluate.load("accuracy")
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.2 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
with open('/content/drive/MyDrive/$th TRY/train_data.json') as f:
    train_json = json.load(f)
with open('/content/drive/MyDrive/$th TRY/val_data.json') as f:
    val_json = json.load(f)
with open('/content/drive/MyDrive/$th TRY/test_data.json') as f:
    test_json = json.load(f)

# Convert to Huggingface Datasets
train_dataset = Dataset.from_list(train_json)
val_dataset = Dataset.from_list(val_json)
test_dataset = Dataset.from_list(test_json)

dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})


In [5]:
model_checkpoint = 'Salesforce/codet5-base'  # Or try 't5-base' or 'google/flan-t5-base'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [6]:
max_input_length = 256
max_target_length = 256

def preprocess_function(examples):
    inputs = examples['input_text']
    targets = examples['target_text']
    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
        padding='max_length'
    )
    labels = tokenizer(
        targets,
        max_length=max_target_length,
        truncation=True,
        padding='max_length'
    )['input_ids']
    labels = [
        [tok if tok != tokenizer.pad_token_id else -100 for tok in seq]
        for seq in labels
    ]
    model_inputs['labels'] = labels
    return model_inputs

tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset['train'].column_names
)


Map:   0%|          | 0/4588 [00:00<?, ? examples/s]

Map:   0%|          | 0/982 [00:00<?, ? examples/s]

Map:   0%|          | 0/979 [00:00<?, ? examples/s]

In [8]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=5e-5,                   # Lower, safer learning rate!
    num_train_epochs=3,                   # Start w/ 3 for stability, increase after
    per_device_train_batch_size=4,        # Small batch
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,        # Effective batch = 16
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="steps",
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    eval_strategy="steps",
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=0,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=3,
    seed=42,
    data_seed=42,
    report_to="none",
)

In [12]:
from transformers.trainer_callback import TrainerCallback

class ValidationPrintCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
        model.eval()
        np.random.seed(42)
        for idx in np.random.choice(len(val_json), size=3, replace=False):
            sample = val_json[idx]
            nlq = sample["input_text"]
            target = sample["target_text"]
            inputs = tokenizer(nlq, return_tensors="pt", truncation=True, padding=True, max_length=256).to(model.device)
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=128, num_beams=4)
            predicted_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
            print(f"NLQ:    {nlq[:80]}...")
            print(f"Target: {target}")
            print(f"Pred:   {predicted_sql}\n{'-'*50}")

callbacks = [ValidationPrintCallback()]


In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=callbacks,
)

/tmp/ipython-input-14-2511318090.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
train_result = trainer.train()
trainer.save_model("./finetuned_clinical_sql_t5")


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
500,0.059900,0.053252


NLQ:    translate to sql: Get patients of Dr. Russell Database Schema: clinical_data
Tab...
Target: SELECT DISTINCT p.first_name, p.last_name FROM clinical_data.patients p JOIN clinical_data.encounters e ON p.id = e.patient_id JOIN clinical_data.providers pr ON e.provider_id = pr.id WHERE pr.name ILIKE '%Russell%' ORDER BY p.last_name
Pred:   SELECT DISTINCT p.first_name, p.last_name FROM clinical_data.patients p JOIN clinical_data.encounters e ON p.id = e.patient_id JOIN clinical_data.providers pr ON e.provider_id = pr.id WHERE pr.name ILIKE '%Russell%' ORDER BY p.last_name
--------------------------------------------------
NLQ:    translate to sql: Show visits to provider Ross Database Schema: clinical_data
Ta...
Target: SELECT p.first_name, p.last_name, e.start_date, e.encounter_class FROM clinical_data.patients p JOIN clinical_data.encounters e ON p.id = e.patient_id JOIN clinical_data.providers pr ON e.provider_id = pr.id WHERE pr.name ILIKE '%Ross%' ORDER BY e.start_date DESC
Pre

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


In [16]:
eval_results = trainer.evaluate(tokenized_datasets["test"])
print("Test set results:", eval_results)


NLQ:    translate to sql: Get patients of Dr. Russell Database Schema: clinical_data
Tab...
Target: SELECT DISTINCT p.first_name, p.last_name FROM clinical_data.patients p JOIN clinical_data.encounters e ON p.id = e.patient_id JOIN clinical_data.providers pr ON e.provider_id = pr.id WHERE pr.name ILIKE '%Russell%' ORDER BY p.last_name
Pred:   SELECT DISTINCT p.first_name, p.last_name FROM clinical_data.patients p JOIN clinical_data.encounters e ON p.id = e.patient_id JOIN clinical_data.providers pr ON e.provider_id = pr.id WHERE pr.name ILIKE '%Russell%' ORDER BY p.last_name
--------------------------------------------------
NLQ:    translate to sql: Show visits to provider Ross Database Schema: clinical_data
Ta...
Target: SELECT p.first_name, p.last_name, e.start_date, e.encounter_class FROM clinical_data.patients p JOIN clinical_data.encounters e ON p.id = e.patient_id JOIN clinical_data.providers pr ON e.provider_id = pr.id WHERE pr.name ILIKE '%Ross%' ORDER BY e.start_date DESC
Pre

In [17]:
def test_model_on_sample(index=0):
    sample = test_json[index]
    nlq = sample["input_text"]
    true_sql = sample["target_text"]
    inputs = tokenizer(nlq, return_tensors="pt", truncation=True, padding=True).to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=256, num_beams=4, no_repeat_ngram_size=3)
    predicted_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"\n🟡 NLQ Input:\n{nlq}\n")
    print(f"✅ Ground Truth SQL:\n{true_sql}\n")
    print(f"🔷 Predicted SQL:\n{predicted_sql}\n")
    print("✅✅ CORRECT!" if predicted_sql.strip() == true_sql.strip() else "❌❌ INCORRECT.")

test_model_on_sample(0)



🟡 NLQ Input:
translate to sql: Get providers in NM state Database Schema: clinical_data
Tables: patients, organizations, providers, encounters, conditions, medications, procedures, observations, allergies, careplans, immunizations, claims, payers
Key relationships: 
- patients.id -> encounters.patient_id
- providers.id -> encounters.provider_id  
- organizations.id -> providers.organization_id
- encounters.id -> conditions.encounter_id
- encounters.id -> medications.encounter_id
- encounters.id -> procedures.encounter_id
- encounters.id -> observations.encounter_id
- payers.id -> claims.payer_id

✅ Ground Truth SQL:
SELECT name, speciality, city FROM clinical_data.providers WHERE state = 'NM' ORDER BY name

🔷 Predicted SQL:
SELECT name, speciality FROM clinical_data.providers WHERE state = 'NM' ORDER BY name

❌❌ INCORRECT.


In [23]:
import shutil

MODEL_DIR = './finetuned_clinical_sql_t5'  # change as needed (e.g. './results/checkpoint-XXXX')
ZIP_PATH = '/content/finetuned_clinical_sql_t5.zip'

shutil.make_archive('/content/finetuned_clinical_sql_t5', 'zip', MODEL_DIR)


'/content/finetuned_clinical_sql_t5.zip'

In [24]:
from google.colab import files
files.download(ZIP_PATH)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
!zip -r finetuned_t5_nl2sql.zip . -i ./content/drive/MyDrive/YOUR_FOLDER/finetuned_clinical_sql_t5

	zip warning: zip file empty


# Again Trainig model for more refinemenets


In [28]:
import os
OUTPUT_ROOT = '/content/drive/MyDrive/$th TRY'
training_args = TrainingArguments(
    output_dir=os.path.join(OUTPUT_ROOT, 'results'),   # Drive path for safety
    learning_rate=5e-5,
    num_train_epochs=3,              # Increase if loss still dropping
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="steps",
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    eval_strategy="steps",
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=0,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=3,
    seed=42,
    data_seed=42,
    report_to="none",
)

In [29]:
from transformers.trainer_callback import TrainerCallback

class ValidationPrintCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
        model.eval()
        np.random.seed(42)
        for idx in np.random.choice(len(val_json), size=3, replace=False):
            sample = val_json[idx]
            nlq = sample["input_text"]
            target = sample["target_text"]
            inputs = tokenizer(nlq, return_tensors="pt", truncation=True, padding=True, max_length=256).to(model.device)
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=128, num_beams=4)
            predicted_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
            print(f"NLQ:    {nlq[:80]}...")
            print(f"Target: {target}")
            print(f"Pred:   {predicted_sql}\n{'-'*50}")

callbacks = [ValidationPrintCallback()]


In [37]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=callbacks
)


/tmp/ipython-input-37-2476638772.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
checkpoint_path = '/content/results/checkpoint-500'  # update this
train_result = trainer.train(resume_from_checkpoint=checkpoint_path)



trainer.save_model(os.path.join(OUTPUT_ROOT, "finetuned_clinical_sql_t5"))


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


Step,Training Loss,Validation Loss


In [ ]:
trainer.save_model("/content/drive/MyDrive/YOUR_FOLDER/finetuned_clinical_sql_t5")


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/YOUR_FOLDER/finetuned_clinical_sql_t5")
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
